In [11]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

In [2]:
class CausalConv1d(nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):
        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)

        self.left_padding = dilation * (kernel_size - 1)

    def forward(self, input):
        x = F.pad(input.unsqueeze(2), (self.left_padding, 0, 0, 0)).squeeze(1)

        return super(CausalConv1d, self).forward(x)

In [3]:
causal = CausalConv1d(4, 4, 2)
print(causal)

CausalConv1d (4, 4, kernel_size=(2,), stride=(1,))


In [4]:
params = list(causal.parameters())
print(params)

[Parameter containing:
(0 ,.,.) = 
  0.1972 -0.1522
  0.0947  0.3348
 -0.2032 -0.2343
  0.0013  0.1489

(1 ,.,.) = 
 -0.1403  0.0836
  0.3381  0.1465
 -0.2728 -0.3497
 -0.0005  0.3177

(2 ,.,.) = 
 -0.2204 -0.3228
 -0.2327 -0.1988
 -0.2115 -0.2861
  0.1699  0.0782

(3 ,.,.) = 
 -0.2387 -0.3519
 -0.0392 -0.0722
 -0.1853 -0.3001
  0.3254  0.0656
[torch.FloatTensor of size 4x4x2]
, Parameter containing:
-0.1365
-0.1545
-0.1244
-0.1619
[torch.FloatTensor of size 4]
]


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(causal.parameters(), lr=0.001, momentum=0.9)

Create the input and target vectors, turn

In [7]:
x = np.array([[1,5,2,3]]) # by placing additional brackets around the array, it goes from size 4 to 1*4
input = torch.from_numpy(x).type(torch.FloatTensor) # turn a numpy array to a torch tensor and specify the type
print(input)

y = np.array([[0,2,10,4]]) # by placing additional brackets around the array, it goes from size 4 to 1*4
target = Variable(torch.from_numpy(y).type(torch.FloatTensor)) # turn a numpy array to a torch tensor and specify the type
print(target)

Variable containing:
  0   2  10   4
[torch.FloatTensor of size 1x4]



In [ ]:
input = Variable(b)
out = causal(input)
print(out)

In [ ]:
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = causal(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

In [ ]:
causal.zero_grad()
out.backward(torch.randn(1, 10))

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(causal.parameters(), lr=0.001, momentum=0.9)